<b> Code pulls current blue book value for my vehicle
    
    
    Assumes zip code and vehicle options do not change
    Went through the process of selecting features beforehand, with this getting added to the webaddress
    

In [60]:
#Check to make sure we don't get an error accessing the page
import requests
res = requests.get('https://www.kbb.com/gmc/terrain/2018/denali-sport-utility-4d/?condition=good&intent=trade-in-sell&mileage=37500&modalview=false&options=8039220%7ctrue%7c8039240%7ctrue%7c8234743%7ctrue%7c8039224%7ctrue%7c8039215%7ctrue%7c8039239%7ctrue%7c8039228%7ctrue%7c8039230%7ctrue%7c8039225%7ctrue&pricetype=private-party&vehicleid=430811')

requests.models.Response

<b>Awesome!! We are able to access the page. Now the features will remain consistent, the only things that could change are the mileage and the condition of the vehicle. I will assume the zip code will remain the same for my purposes
    
Say I were to change the mileage, condition, and sale type. The web address now changes from this:
https://www.kbb.com/gmc/terrain/2018/denali-sport-utility-4d/?condition=verygood&intent=trade-in-sell&mileage=37500&modalview=false&options=8039220%7ctrue%7c8039240%7ctrue%7c8234743%7ctrue%7c8039224%7ctrue%7c8039215%7ctrue%7c8039239%7ctrue%7c8039228%7ctrue%7c8039230%7ctrue%7c8039225%7ctrue&pricetype=trade-in&vehicleid=430811
    
https://www.kbb.com/gmc/terrain/2018/denali-sport-utility-4d/?condition=excellent&intent=trade-in-sell&mileage=9999&modalview=false&options=8039220%7ctrue%7c8039240%7ctrue%7c8234743%7ctrue%7c8039224%7ctrue%7c8039215%7ctrue%7c8039239%7ctrue%7c8039228%7ctrue%7c8039230%7ctrue%7c8039225%7ctrue&pricetype=private-party&vehicleid=430811
    
So to change our inputs we just have to change the address. Now I just need to find where to scrape my values from.

In [6]:
#make a soup from my request
import bs4
soup = bs4.BeautifulSoup(res.text, 'lxml')

<b>Beautiful Soup seems to not have any luck as it recognizes me a as bot. Let's try using regex to see if we can't dig out the price estimates from res.text


In [65]:
res.text

'<!doctype html><html lang="en"><head><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=5,minimum-scale=1"><meta http-equiv="x-dns-prefetch-control" content="on"><link rel="dns-prefetch preconnect" href="//securepubads.g.doubleclick.net" crossorigin><link rel="dns-prefetch preconnect" href="//c.amazon-adsystem.com" crossorigin><link rel="dns-prefetch preconnect" href="//ads.pubmatic.com" crossorigin><link rel="dns-prefetch preconnect" href="//www.google.com" crossorigin><link rel="dns-prefetch preconnect" href="//adservice.google.com" crossorigin><link rel="dns-prefetch preconnect" href="//mb.moatads.com" crossorigin><link rel="dns-prefetch preconnect" href="//googleads.g.doubleclick.net" crossorigin><link rel="dns-prefetch preconnect" href="//assets.adobedtm.com" crossorigin><link rel="dns-prefetch preconnect" href="//fonts.gstatic.com" crossorigin><link rel="dns-prefetch preconnec

In [62]:
import re

price = re.search('\$\d{2},\d{3}', res.text)

In [63]:
price.group()

AttributeError: 'NoneType' object has no attribute 'group'

<b> Ok not necessarily the right way, or the most efficient way, but seems to do the job!! Now I will combine it all into a function

In [64]:
import re, requests
def davy_car_kbb(mileage,condition):
    price_type = ('trade-in','private-party')
   
    for typ in price_type:
        url = f'https://www.kbb.com/gmc/terrain/2018/denali-sport-utility-4d/?condition={condition}&intent=trade-in-sell&mileage={mileage}&modalview=false&options=8039220%7ctrue%7c8039240%7ctrue%7c8234743%7ctrue%7c8039224%7ctrue%7c8039215%7ctrue%7c8039239%7ctrue%7c8039228%7ctrue%7c8039230%7ctrue%7c8039225%7ctrue&pricetype={typ}&vehicleid=430811'
        res = requests.get(url, 'lxml')
        price = re.search('\$\d{2},\d{3}', res.text)
        
        if typ == 'trade-in':
            trade_in = price.group()
        if typ == 'private-party':
            #print(res.text)
            pass
         #   private_party = price.group()
    
    return trade_in
    #print(f'Trade-in Value: {trade_in}')
    #print(f'Private Party Sale Value: {private_party}')
          

Condition has to be either excellent, verygood, good, or fair. KBB doesn't evaluate poor

In [74]:
#Test it!!

davy_car_kbb(100000, 'good')

Trade-in Value: $20,955


<b> Code works for the trade-in, but not the private sale. I can't figure out why exactly, because the generated link works correctly and displays the price. However, it doesn't show up in res.text
    
Next let's set up a simple GUI to determine the trade-in value of the vehicle from KBB

In [62]:
from IPython.display import display
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [71]:
#Set up Widgets
mileage_widget = widgets.IntText(value=30000)
condition_widget = widgets.ToggleButtons(options=['Excellent', 'Very Good', 'Good', 'Fair'])
button = widgets.Button(description = "Get KBB Trade-In Value")
value_widget = widgets.HTML(value= '<b>Trade-In Value: <b>')

#Some basic styling
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='95%')
condition_widget.layout = button.layout = mileage_widget.layout = box_layout
button.style.button_color = 'lightblue'
button.style.font_weight = 'bold'
display(mileage_widget,condition_widget,button,value_widget)


def on_button_clicked(b):
    value_widget.value = ''
    condition = condition_widget.value.lower().replace(' ','')
    value = davy_car_kbb(mileage_widget.value, condition)
    value_widget.value = '<b>Trade-In Value: <b>' + value

button.on_click(on_button_clicked)

IntText(value=30000, layout=Layout(align_items='center', display='flex', flex_flow='column', width='95%'))

ToggleButtons(layout=Layout(align_items='center', display='flex', flex_flow='column', width='95%'), options=('…

Button(description='Get KBB Trade-In Value', layout=Layout(align_items='center', display='flex', flex_flow='co…

HTML(value='<b>Trade-In Value: <b>')